<a href="https://colab.research.google.com/github/Aayush-Jain01/Object-Detection/blob/main/Object_Detection_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ray[tune]

     |████████████████████████████████| 53.2 MB 197 kB/s 
     |████████████████████████████████| 144 kB 50.0 MB/s 
     |████████████████████████████████| 8.8 MB 38.6 MB/s 
     |████████████████████████████████| 4.1 MB 22.0 MB/s 
     |████████████████████████████████| 125 kB 51.2 MB/s 
     |████████████████████████████████| 461 kB 51.6 MB/s 
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.44.0
    Uninstalling grpcio-1.44.0:
      Successfully uninstalled grpcio-1.44.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.


In [2]:
import torch
import torchvision
from torch import nn
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torch.utils.data import random_split
import numpy as np
from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
import os
import pandas as pd
from skimage import io, transform
from PIL import Image
import cv2
import math
import sys
import time
import io as F 
import datetime
import errno
from collections import defaultdict, deque
import torch.distributed as dist
from contextlib import redirect_stdout
from collections import defaultdict, deque
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection import fasterrcnn_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator

In [3]:
 from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
class CarDataset(Dataset):
  def __init__(self, csv_file, root_dir, transform):
    super().__init__()
    self.csv_file = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform
  def __len__(self):
    return len(self.csv_file)
  def __getitem__(self, idx):
    img_path = os.path.join(self.root_dir, self.csv_file.iloc[idx, 0])
    image = io.imread(img_path)                     #Reading the image
    image_ = Image.fromarray(image)                 #Converting image to PIL to be able to apply the transforms
    bounding_params = list(self.csv_file.iloc[idx, 1:])   
    #bounding_params = np.array([bounding_params])
    bounding_params = torch.as_tensor(bounding_params, dtype= torch.float32)
    bounding_params = torch.reshape(bounding_params, (1, 4))
    labels = torch.ones((1, ), dtype= torch.int64)
    image_id = torch.tensor([idx])
    area = torch.as_tensor([(bounding_params[0][3]-bounding_params[0][1])*(bounding_params[0][2]-bounding_params[0][0])])
    iscrowd = torch.zeros((1, ), dtype=torch.int64)
    target = {}
    target["boxes"] = bounding_params
    target["labels"] = labels
    target["image_id"] = image_id
    target["area"] = area
    target["iscrowd"] = iscrowd
    #sample = {'image': image, "bounding_parms" : bounding_params}
    if self.transform:
        image_1 = self.transform(image_)
    return image_1, target


In [5]:
train_transforms = transforms.Compose([
        #transforms.RandomResizedCrop(size=256, scale=(0.8, 1.0)),
        #transforms.RandomRotation(degrees=15),
        #transforms.ColorJitter(0.3, 0.3, 0.3, 0.3),
        # transforms.RandomHorizontalFlip(0.5),
        #transforms.CenterCrop(size=224),  # Image net standards
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406],
                             #[0.229, 0.224, 0.225])  # Imagenet standards
      ])
test_transforms = transforms.Compose([
        #transforms.Resize(size=256),
        #transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        #transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
train_set = CarDataset("/content/drive/MyDrive/YOLO Object Detection/train_solution_bounding_boxes (1).csv", "/content/drive/MyDrive/YOLO Object Detection/training_images/", transform= train_transforms)
train_set

In [6]:
train_set.__getitem__(0)

(tensor([[[0.5020, 0.4980, 0.4980,  ..., 0.3922, 0.3922, 0.3882],
          [0.5020, 0.5020, 0.4980,  ..., 0.4000, 0.4000, 0.4000],
          [0.5059, 0.5059, 0.5020,  ..., 0.4078, 0.4078, 0.4118],
          ...,
          [0.1020, 0.1098, 0.1059,  ..., 0.0824, 0.0824, 0.0824],
          [0.0941, 0.1059, 0.1059,  ..., 0.0824, 0.0824, 0.0824],
          [0.0980, 0.1098, 0.1098,  ..., 0.0784, 0.0784, 0.0784]],
 
         [[0.8275, 0.8235, 0.8235,  ..., 0.7255, 0.7176, 0.7137],
          [0.8275, 0.8275, 0.8235,  ..., 0.7137, 0.7137, 0.7137],
          [0.8314, 0.8314, 0.8275,  ..., 0.7137, 0.7137, 0.7098],
          ...,
          [0.1922, 0.2000, 0.2000,  ..., 0.1686, 0.1686, 0.1686],
          [0.1843, 0.1961, 0.2000,  ..., 0.1686, 0.1686, 0.1686],
          [0.1882, 0.2000, 0.2039,  ..., 0.1647, 0.1647, 0.1647]],
 
         [[1.0000, 0.9961, 0.9961,  ..., 0.9804, 0.9765, 0.9725],
          [1.0000, 1.0000, 0.9961,  ..., 0.9922, 0.9922, 0.9922],
          [0.9961, 0.9961, 0.9922,  ...,

In [7]:
def collate_func(batch):
    return tuple(zip(*batch))
train_set, val_set = random_split(train_set, lengths=[447,112])
test_set = CarDataset("/content/drive/MyDrive/YOLO Object Detection/sample_submission.csv", "/content/drive/MyDrive/YOLO Object Detection/testing_images", test_transforms)
train_dl =  DataLoader(train_set, batch_size= 3, collate_fn= collate_func, shuffle = True)
val_dl = DataLoader(val_set, batch_size= 4, collate_fn= collate_func, shuffle = True)
test_dl = DataLoader(test_set, batch_size = 4, collate_fn = collate_func, shuffle = False)


In [8]:
from torchvision import models


In [9]:
model = models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
for parameters in model.parameters():
  parameters.requires_grad = False
num_classes = 2 
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


  0%|          | 0.00/160M [00:00<?, ?B/s]

In [10]:
model.to('cuda')

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [11]:
%%bash
git clone https://github.com/pytorch/vision.git
cd vision
git checkout v0.3.0
cp references/detection/utils.py ../
cp references/detection/transforms.py ../
cp references/detection/coco_eval.py ../
cp references/detection/engine.py ../
cp references/detection/coco_utils.py ../

Cloning into 'vision'...
Note: checking out 'v0.3.0'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at be376084d version check against PyTorch's CUDA version


In [12]:
from engine import train_one_epoch, evaluate
import utils

In [13]:
device = torch.device('cuda')
# our dataset has two classes only - raccoon and not racoon
num_classes = 2
# get the model using our helper function
# move model to the right device
model.to(device)
# construct an optimizer
params = [p for p in model.parameters() if p.requires_grad]

In [14]:

def eval_one_epoch(model, optimizer, data_loader, device, epoch, print_freq):
    model.eval()
    print("---------Evaluating-----------")
    metric_logger = utils.MetricLogger(delimiter="  ")
    metric_logger.add_meter("lr", utils.SmoothedValue(window_size=1, fmt="{value:.6f}"))
    header = "Epoch: [{}]".format(epoch)

    lr_scheduler = None
    # if epoch == 0:
    #     warmup_factor = 1.0 / 1000
    #     warmup_iters = min(1000, len(data_loader) - 1)

    #     lr_scheduler = torch.optim.lr_scheduler.LinearLR(
    #         optimizer, start_factor=warmup_factor, total_iters=warmup_iters
    #     )

    for images, targets in metric_logger.log_every(data_loader, print_freq, header):
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)

        losses = sum(loss for loss in loss_dict.values())

        # reduce losses over all GPUs for logging purposes
        loss_dict_reduced = utils.reduce_dict(loss_dict)
        losses_reduced = sum(loss for loss in loss_dict_reduced.values())

        loss_value = losses_reduced.item()

        if not math.isfinite(loss_value):
            print("Loss is {}, stopping training".format(loss_value))
            print(loss_dict_reduced)
            sys.exit(1)

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        if lr_scheduler is not None:
            lr_scheduler.step()

        metric_logger.update(loss=losses_reduced, **loss_dict_reduced)
        metric_logger.update(lr=optimizer.param_groups[0]["lr"])

    return loss_value

In [15]:
# def train(config, checkpoint_dir=None, data_dir=None):
#     optimizer = torch.optim.Adam(params, config["lr"], betas= (config["b1"], config["b2"]))
#     # and a learning rate scheduler which decreases the learning rate by # 10x every 3 epochs
#     #lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
#     #lr_schedular = torch.optim.lr_scheduler.CyclicLR(optimizer, config["base_lr"], config["max_lr"], gamma=config["gamma"]
#     if checkpoint_dir:
#         model_state, optimizer_state = torch.load(
#             os.path.join(checkpoint_dir, "checkpoint"))
#         model.load_state_dict(model_state)
#         optimizer.load_state_dict(optimizer_state)

#     train_dl =  DataLoader(train_set, batch_size= int(config["batch_size"]), collate_fn= collate_func, shuffle = True, num_workers=8)
#     val_dl = DataLoader(val_set, batch_size= int(config["batch_size"]), collate_fn= collate_func, shuffle = True, num_workers=8)
#     test_dl = DataLoader(test_set, batch_size= int(config["batch_size"]), collate_fn = collate_func, shuffle = False)

#     for epoch in range(10):  # loop over the dataset multiple times
#         train_one_epoch(model, optimizer, train_dl, device, epoch,
#                       print_freq=10)

#         #lr_scheduler.step()

#         a = eval_one_epoch(model, optimizer, val_dl, device, epoch,
#                       print_freq=10)
        
#         with tune.checkpoint_dir(epoch) as checkpoint_dir:
#             path = os.path.join(checkpoint_dir, "checkpoint")
#             torch.save((model.state_dict(), optimizer.state_dict()), path)

#         tune.report(loss= a)
#     print("Finished Tuning !")

In [16]:
def train(config):
    # optimizer = torch.optim.Adam(params, config["lr"], betas= (config["b1"], config["b2"]))
    optimizer = torch.optim.SGD(model.parameters(), lr= config["lr"], weight_decay= config["weight_decay"])
    lr_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr= config["base_lr"], max_lr = config["max_lr"], gamma= config["gamma"]) 
    # and a learning rate scheduler which decreases the learning rate by # 10x every 3 epochs
    #lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)
    # lr_schedular = torch.optim.lr_scheduler.CyclicLR(optimizer, config["base_lr"], config["max_lr"], gamma=config["gamma"])
    num_epochs = 50
    for epoch in range(num_epochs):
      # train for one epoch, printing every 10 iterations
      train_one_epoch(model, optimizer, train_dl, device, epoch,
                      print_freq=10)
    # update the learning rate
      lr_scheduler.step()
      # evaluate on the test dataset
      evaluate(model, val_dl, device=device)

In [17]:
config = {
    # "lr" : 1e-5,
    # "b1" : 0.9,
    # "b2" : 0.99,
    # "batch_size" : 4,
    # "base_lr" : 1e-5,
    # "max_lr" : 5e-3,
    # "gamma" : 1.0
    "lr" : 1e-4,
    "weight_decay" : 1e-5,
    "base_lr" : 1e-5,
    "max_lr" : 5e-3,
    "gamma" : 1.0}
train(config)

/usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


Epoch: [0]  [  0/149]  eta: 0:08:08  lr: 0.000000  loss: 1.0311 (1.0311)  loss_classifier: 0.8948 (0.8948)  loss_box_reg: 0.1318 (0.1318)  loss_objectness: 0.0011 (0.0011)  loss_rpn_box_reg: 0.0034 (0.0034)  time: 3.2784  data: 1.9172  max mem: 1388
Epoch: [0]  [ 10/149]  eta: 0:06:28  lr: 0.000001  loss: 0.9936 (0.9849)  loss_classifier: 0.8448 (0.8458)  loss_box_reg: 0.1292 (0.1258)  loss_objectness: 0.0092 (0.0094)  loss_rpn_box_reg: 0.0033 (0.0038)  time: 2.7968  data: 1.7374  max mem: 1388
Epoch: [0]  [ 20/149]  eta: 0:05:39  lr: 0.000001  loss: 0.9883 (0.9889)  loss_classifier: 0.8448 (0.8474)  loss_box_reg: 0.1292 (0.1275)  loss_objectness: 0.0104 (0.0110)  loss_rpn_box_reg: 0.0020 (0.0030)  time: 2.6034  data: 1.5756  max mem: 1388
Epoch: [0]  [ 30/149]  eta: 0:05:00  lr: 0.000002  loss: 0.9853 (0.9879)  loss_classifier: 0.8465 (0.8470)  loss_box_reg: 0.1234 (0.1258)  loss_objectness: 0.0105 (0.0115)  loss_rpn_box_reg: 0.0026 (0.0036)  time: 2.3770  data: 1.3601  max mem: 1388


In [ ]:
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
    config = {
        "lr" : tune.loguniform(5e-4, 1e-3),
        "b1" : tune.loguniform(0.9, 1),
        "b2" : tune.loguniform(0.9, 1),
        "base_lr" : tune.loguniform(1e-4, 1e-6),
        "max_lr" : tune.loguniform(1e-2, 1e-4),
        "gamma" : 1.0,
        "batch_size" : tune.choice([2, 4, 8, 16]),
    }
    scheduler = ASHAScheduler(
        metric= "loss",
        mode= "min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        # parameter_columns=["l1", "l2", "lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        run_or_experiment = train,
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))
    
    # best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
    # device = "cpu"
    # if torch.cuda.is_available():
    #     device = "cuda:0"
    #     if gpus_per_trial > 1:
    #         best_trained_model = nn.DataParallel(best_trained_model)
    # best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(best_checkpoint_dir, "checkpoint"))
    model.load_state_dict(model_state)
if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=1)